# SQLite Basics in Python

## Learning Objectives

By the end of this notebook, you will be able to:

1. Explain what SQL is and why it's used for data management
2. Understand SQLite's advantages as an embedded database
3. Connect to SQLite databases using Python's sqlite3 module
4. Create both in-memory and file-based databases
5. Create tables with appropriate data types
6. Understand SQLite's type system and how it differs from other databases

## What is SQL?

**SQL (Structured Query Language)** is a standard language for managing and manipulating relational databases. It allows you to:

- **Create** database structures (tables, indexes)
- **Read** data from tables using queries
- **Update** existing records
- **Delete** records from tables

### Why Use SQL?

| Feature | Benefit |
|---------|--------|
| **Declarative** | You describe *what* you want, not *how* to get it |
| **Standardized** | Works across different database systems |
| **Efficient** | Optimized for large datasets |
| **ACID Compliant** | Ensures data integrity (Atomicity, Consistency, Isolation, Durability) |
| **Relational** | Links related data across tables |

## SQLite Overview

**SQLite** is a lightweight, serverless, self-contained SQL database engine. It's:

- **Built into Python** - No installation needed (comes with Python's standard library)
- **Serverless** - No separate server process to manage
- **File-based** - Entire database is stored in a single file
- **Zero configuration** - No setup or administration required
- **Cross-platform** - Database files work on any operating system

### When to Use SQLite

SQLite is ideal for:
- Local data storage in applications
- Prototyping before moving to larger databases
- Data analysis and exploration
- Testing database code
- Embedded systems and mobile apps

For high-concurrency web applications or very large datasets, consider PostgreSQL or MySQL instead.

## Connecting to Databases

Python's `sqlite3` module provides the interface to SQLite databases.

In [ ]:
import sqlite3

# Check the SQLite version
print(f"SQLite version: {sqlite3.sqlite_version}")
print(f"sqlite3 module version: {sqlite3.version}")

### Creating an In-Memory Database

In-memory databases are temporary and exist only while the connection is open. They're great for testing and temporary data processing.

In [ ]:
# Create an in-memory database
conn = sqlite3.connect(':memory:')
print(f"Connection type: {type(conn)}")
print("In-memory database created successfully!")

# Always close connections when done
conn.close()
print("Connection closed.")

### Creating a File-Based Database

File-based databases persist data to disk. If the file doesn't exist, SQLite creates it.

In [ ]:
import os

# Create a file-based database
db_path = 'company.db'

# Remove if it exists (for a fresh start)
if os.path.exists(db_path):
    os.remove(db_path)

conn = sqlite3.connect(db_path)
print(f"Database file created: {os.path.exists(db_path)}")
print(f"File size: {os.path.getsize(db_path)} bytes")

conn.close()

### Using Context Managers (Recommended)

The `with` statement automatically handles closing connections and committing/rolling back transactions.

In [ ]:
# Recommended: Use context manager for automatic cleanup
with sqlite3.connect(':memory:') as conn:
    print("Inside context manager - connection is open")
    # Do database operations here
    
print("Outside context manager - connection auto-closed")

## The Cursor Object

A **cursor** is used to execute SQL statements and fetch results. Think of it as a pointer that moves through the result set.

In [ ]:
with sqlite3.connect(':memory:') as conn:
    # Create a cursor
    cursor = conn.cursor()
    print(f"Cursor type: {type(cursor)}")
    
    # Execute a simple query
    cursor.execute("SELECT sqlite_version()")
    
    # Fetch the result
    result = cursor.fetchone()
    print(f"SQLite version from query: {result[0]}")

## Creating Tables with CREATE TABLE

Tables are created using the `CREATE TABLE` statement. Each column has a name and data type.

In [ ]:
# Create our company database with tables
with sqlite3.connect('company.db') as conn:
    cursor = conn.cursor()
    
    # Create the departments table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS departments (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            budget REAL
        )
    ''')
    
    print("Departments table created!")
    
    # Commit the changes
    conn.commit()

### Understanding the CREATE TABLE Syntax

```sql
CREATE TABLE IF NOT EXISTS table_name (
    column1 DATA_TYPE CONSTRAINTS,
    column2 DATA_TYPE CONSTRAINTS,
    ...
)
```

Common constraints:
- `PRIMARY KEY` - Unique identifier for each row
- `NOT NULL` - Column cannot contain NULL values
- `UNIQUE` - All values must be different
- `DEFAULT value` - Default value if none provided
- `FOREIGN KEY` - References another table's primary key

In [ ]:
# Create all our tables
with sqlite3.connect('company.db') as conn:
    cursor = conn.cursor()
    
    # Drop existing tables to start fresh
    cursor.execute('DROP TABLE IF EXISTS projects')
    cursor.execute('DROP TABLE IF EXISTS employees')
    cursor.execute('DROP TABLE IF EXISTS departments')
    
    # Create departments table
    cursor.execute('''
        CREATE TABLE departments (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL UNIQUE,
            budget REAL DEFAULT 0
        )
    ''')
    
    # Create employees table with foreign key
    cursor.execute('''
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            department_id INTEGER,
            salary REAL,
            hire_date TEXT,
            FOREIGN KEY (department_id) REFERENCES departments(id)
        )
    ''')
    
    # Create projects table
    cursor.execute('''
        CREATE TABLE projects (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            department_id INTEGER,
            start_date TEXT,
            end_date TEXT,
            FOREIGN KEY (department_id) REFERENCES departments(id)
        )
    ''')
    
    conn.commit()
    print("All tables created successfully!")

### Viewing Table Structure

You can query SQLite's system tables to see what tables exist and their structure.

In [ ]:
with sqlite3.connect('company.db') as conn:
    cursor = conn.cursor()
    
    # List all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    
    print("Tables in database:")
    for table in tables:
        print(f"  - {table[0]}")

In [ ]:
with sqlite3.connect('company.db') as conn:
    cursor = conn.cursor()
    
    # Get column information for employees table
    cursor.execute("PRAGMA table_info(employees)")
    columns = cursor.fetchall()
    
    print("Employees table structure:")
    print(f"{'ID':<4} {'Name':<15} {'Type':<10} {'NotNull':<8} {'Default':<10} {'PK'}")
    print("-" * 55)
    for col in columns:
        print(f"{col[0]:<4} {col[1]:<15} {col[2]:<10} {col[3]:<8} {str(col[4]):<10} {col[5]}")

## SQLite Data Types

SQLite uses **dynamic typing** with five storage classes:

| Storage Class | Description | Python Type |
|---------------|-------------|-------------|
| **NULL** | Null value | `None` |
| **INTEGER** | Signed integer (up to 8 bytes) | `int` |
| **REAL** | Floating-point number | `float` |
| **TEXT** | Text string (UTF-8, UTF-16BE or UTF-16LE) | `str` |
| **BLOB** | Binary data | `bytes` |

### Type Affinity

SQLite is flexible - you can store any value in any column (except `INTEGER PRIMARY KEY`). The declared type is a *hint* for how data should be stored.

In [ ]:
# Demonstrating SQLite's dynamic typing
with sqlite3.connect(':memory:') as conn:
    cursor = conn.cursor()
    
    # Create a table with an INTEGER column
    cursor.execute('CREATE TABLE test (value INTEGER)')
    
    # SQLite accepts different types in the same column!
    cursor.execute("INSERT INTO test VALUES (42)")
    cursor.execute("INSERT INTO test VALUES ('hello')")
    cursor.execute("INSERT INTO test VALUES (3.14)")
    cursor.execute("INSERT INTO test VALUES (NULL)")
    
    cursor.execute("SELECT value, typeof(value) FROM test")
    results = cursor.fetchall()
    
    print("Value          | Type")
    print("-" * 25)
    for row in results:
        print(f"{str(row[0]):<14} | {row[1]}")

### Working with Dates

SQLite doesn't have a native DATE type. Dates are typically stored as:
- **TEXT** in ISO format: `'2024-01-15'`
- **INTEGER** as Unix timestamp
- **REAL** as Julian day numbers

SQLite provides built-in date/time functions for TEXT dates.

In [ ]:
with sqlite3.connect(':memory:') as conn:
    cursor = conn.cursor()
    
    # SQLite date functions
    cursor.execute("SELECT date('now')")
    print(f"Current date: {cursor.fetchone()[0]}")
    
    cursor.execute("SELECT datetime('now')")
    print(f"Current datetime: {cursor.fetchone()[0]}")
    
    cursor.execute("SELECT date('now', '+7 days')")
    print(f"One week from now: {cursor.fetchone()[0]}")
    
    cursor.execute("SELECT date('now', '-1 month')")
    print(f"One month ago: {cursor.fetchone()[0]}")

### Python Date Integration

You can configure sqlite3 to automatically convert between Python dates and SQLite text.

In [ ]:
import datetime

# Enable automatic date detection
conn = sqlite3.connect(':memory:', detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES)
cursor = conn.cursor()

# Create table with date column
cursor.execute('CREATE TABLE events (name TEXT, event_date DATE)')

# Insert using Python date object
today = datetime.date.today()
cursor.execute('INSERT INTO events VALUES (?, ?)', ('Meeting', today))

# Retrieve - automatically converts back to Python date
cursor.execute('SELECT name, event_date FROM events')
row = cursor.fetchone()
print(f"Event: {row[0]}")
print(f"Date: {row[1]}")
print(f"Date type: {type(row[1])}")

conn.close()

## Summary

In this notebook, you learned:

1. **SQL** is a standard language for managing relational databases
2. **SQLite** is a lightweight, serverless database built into Python
3. **Connection** - Use `sqlite3.connect()` with `:memory:` or a file path
4. **Cursor** - Execute SQL and fetch results with cursor objects
5. **CREATE TABLE** - Define tables with columns, types, and constraints
6. **Data Types** - SQLite uses 5 storage classes (NULL, INTEGER, REAL, TEXT, BLOB)
7. **Best Practice** - Use context managers (`with`) for automatic cleanup

### Key SQL Statements Covered

```sql
-- Create a table
CREATE TABLE table_name (
    column_name DATA_TYPE CONSTRAINTS
);

-- List tables
SELECT name FROM sqlite_master WHERE type='table';

-- View table structure  
PRAGMA table_info(table_name);
```

## Exercises

### Exercise 1: Create an In-Memory Database

Create an in-memory database connection and verify it works by selecting the SQLite version.

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
import sqlite3

with sqlite3.connect(':memory:') as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT sqlite_version()')
    version = cursor.fetchone()[0]
    print(f"SQLite version: {version}")
```
</details>

### Exercise 2: Create a Products Table

Create a table called `products` with the following columns:
- `id` - INTEGER, primary key
- `name` - TEXT, not null
- `price` - REAL
- `quantity` - INTEGER, default 0

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
with sqlite3.connect(':memory:') as conn:
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE products (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            price REAL,
            quantity INTEGER DEFAULT 0
        )
    ''')
    
    # Verify the table was created
    cursor.execute("PRAGMA table_info(products)")
    for col in cursor.fetchall():
        print(col)
```
</details>

### Exercise 3: Create Related Tables

Create two related tables:
1. `authors` - id (primary key), name (text, not null)
2. `books` - id (primary key), title (text, not null), author_id (foreign key to authors)

Then list all tables in the database.

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
with sqlite3.connect(':memory:') as conn:
    cursor = conn.cursor()
    
    # Create authors table
    cursor.execute('''
        CREATE TABLE authors (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL
        )
    ''')
    
    # Create books table with foreign key
    cursor.execute('''
        CREATE TABLE books (
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            author_id INTEGER,
            FOREIGN KEY (author_id) REFERENCES authors(id)
        )
    ''')
    
    # List all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    print("Tables created:")
    for table in tables:
        print(f"  - {table[0]}")
```
</details>

### Exercise 4: Explore Date Functions

Using SQLite's date functions:
1. Get the current date
2. Get the date 30 days from now
3. Get the first day of the current month

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
with sqlite3.connect(':memory:') as conn:
    cursor = conn.cursor()
    
    # Current date
    cursor.execute("SELECT date('now')")
    print(f"Current date: {cursor.fetchone()[0]}")
    
    # 30 days from now
    cursor.execute("SELECT date('now', '+30 days')")
    print(f"30 days from now: {cursor.fetchone()[0]}")
    
    # First day of current month
    cursor.execute("SELECT date('now', 'start of month')")
    print(f"First of month: {cursor.fetchone()[0]}")
```
</details>

### Exercise 5: Check Data Types

Create a table and insert values of different types into a single column. Use `typeof()` to check what SQLite actually stored.

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
with sqlite3.connect(':memory:') as conn:
    cursor = conn.cursor()
    
    cursor.execute('CREATE TABLE type_test (data TEXT)')
    
    # Insert various types
    cursor.execute("INSERT INTO type_test VALUES (100)")
    cursor.execute("INSERT INTO type_test VALUES (3.14159)")
    cursor.execute("INSERT INTO type_test VALUES ('Hello')")
    cursor.execute("INSERT INTO type_test VALUES (NULL)")
    cursor.execute("INSERT INTO type_test VALUES (X'0102030405')")
    
    cursor.execute("SELECT data, typeof(data) FROM type_test")
    print("Value              | Type")
    print("-" * 30)
    for row in cursor.fetchall():
        print(f"{str(row[0]):<18} | {row[1]}")
```
</details>

## Next Steps

In the next notebook, **02_crud_operations.ipynb**, you'll learn how to:
- Insert data into tables (single and multiple rows)
- Query data with SELECT statements
- Update existing records
- Delete records
- Use parameters to prevent SQL injection
- Manage transactions with commit and rollback

## Cleanup

Remove the database file created during this notebook.

In [ ]:
import os

if os.path.exists('company.db'):
    os.remove('company.db')
    print("Database file removed.")